In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

#!pip install ray
#!pip install -U tensorboardx

import numpy as np 
import pandas as pd 
from functools import partial
import os
from filelock import FileLock

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST
from torchmetrics import Accuracy

from ray import air, tune
from ray.air import Checkpoint, session
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title old net
class Net(nn.Module):
    def __init__(self, l1=12, l2=10,p=0.2):
        super(Net, self).__init__()
        self.fc1=nn.Linear(784, l1) # 16 input features, 12 output features also called neurons
        self.fc2=nn.Linear(l1, l2)
        self.out=nn.Linear(l2, 10)
        self.m = nn.LogSoftmax(dim=0)
        self.drop = nn.Dropout(p)
    
    def forward(self, x):
        x=x.squeeze()
        x = self.fc1(x)
        x = F.relu(x)
        x = self.drop(x)
       
        x = self.fc2(x)
        x = F.relu(x)
        x = self.drop(x)
       
        x=self.out(x)
        x = F.relu(x)
        #x = self.drop(x)
        output = self.m(x)
        
        return output


In [ ]:
#@title og load data
def load_data(data_dir="/kaggle/working/"):
    training_data = torchvision.datasets.FashionMNIST(
    root="data_dir",
    train=True,
    download=True,
    transform=transforms.ToTensor()
    )

    test_data = torchvision.datasets.FashionMNIST(
    root="data_dir",
    train=False,
    download=True,
    transform=transforms.ToTensor()
    )
    training_data.data=torch.flatten(training_data.data, start_dim=1)
    test_data.data=torch.flatten(test_data.data, start_dim=1)
    #print(training_data.data.shape)
    return training_data, test_data
#t,te=load_data()
#print(t.data.shape)
#print(te.data.shape)

torch.Size([60000, 784])
torch.Size([10000, 784])


In [ ]:
#@title accuracy
def test_accuracy(net, device="cpu"):
    _, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        c=0
        for data in testloader:
                  #  c+=1
                    #if (c==2500):
                       # print('g',c)
                    #    break
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    #print(images.shape)
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
    return correct / total
#print(test_accuracy(Net(16,8)))
#print(test_accuracy(Net(16,8)))
#print(test_accuracy(Net(16,8)))

In [ ]:
def train_cifar(config,epochs=10,data_dir=None,trainset=None,testset=None,tune=False):
    net = Net(l1=config["l1"],l2=config["l2"],p=config["dropout"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
            
    net.to(device)
    criterion = nn.CrossEntropyLoss()

    if config["opt"]=="SGD":
        #if tune:
           # config["p"]=np.random.uniform(low=.85, high=.95)
        optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=.9)
        
    elif config["opt"]=="Adam":
        optimizer = optim.Adam(net.parameters(),lr=0)# lr=config["lr"])
        
    start_epoch = 0
    if tune:
        checkpoint = session.get_checkpoint()

        if checkpoint:
            checkpoint_state = checkpoint.to_dict()
            start_epoch = checkpoint_state["epoch"]
            net.load_state_dict(checkpoint_state["net_state_dict"])
            optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])

    
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2
    )

    for epoch in range(start_epoch, epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = net(inputs)
            
            #print(labels)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
           

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        if tune:
            checkpoint_data = {
                       "epoch": epoch,
                        "net_state_dict": net.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
            }
            checkpoint = Checkpoint.from_dict(checkpoint_data)
            session.report(
                        {"loss": val_loss / val_steps, "accuracy": correct / total},
                        checkpoint=checkpoint,
                )
            """
            if config["opt"]=="SGD":
                session.report(
                        {"loss": val_loss / val_steps, "accuracy": correct / total},
                        checkpoint=checkpoint,
                )
            elif config["opt"]=="Adam":
                session.report(
                        {"loss": 50, "accuracy": correct / total},
                        checkpoint=checkpoint,
                )
              """
        else:
            if epoch%5==0:
                print("epoch,val loss,%correct",epoch,val_loss,correct/total)
    print("Finished Training")
    
    return net

In [ ]:
#@title hp3
def hyperparam_search2(config,scheduler="ASHA",cpu=4,gpu=0,max_num_epochs=10,concurrent=1):
   #determins how the searching works
    if scheduler=="ASHA":
        #tries various permutations of parameters
        sched = ASHAScheduler(
            #metric="loss",
            #mode="min",
            #mode="max",
            max_t=max_num_epochs,
            grace_period=1,
            reduction_factor=2,
        )
    
    if scheduler=="PB2":
        pass
    
    trainset, testset = load_data()  

    tuner = tune.Tuner(
        tune.with_parameters(train_cifar,trainset=trainset,testset=testset,tune=True),
        param_space=config,
        tune_config=tune.TuneConfig(
           # num_samples=num_samples, different when using grid search
            metric="loss",
            mode="min",
            scheduler=sched,
            max_concurrent_trials=concurrent
            )
    )
       
    results = tuner.fit()

    
    best_trial = results.get_best_result(metric="loss", mode="min")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.metrics['accuracy']}")
    
    return best_trial.config
    

In [ ]:
#@title og hps
def hyperparam_search(config,scheduler="ASHA",cpu=4,gpu=0,num_samples=20,max_num_epochs=10):
    
    #determins how the searching works
    if scheduler=="ASHA":
        #tries various permutations of parameters
        scheduler = ASHAScheduler(
            metric="loss",
            #mode="min",
            mode="max",
            max_t=max_num_epochs,
            grace_period=1,
            reduction_factor=2,
        )
    
    if scheduler=="PB2":
        pass
    
    
    trainset, testset = load_data()
    
    
    result = tune.run(
        partial(train_cifar,trainset=trainset,testset=testset,tune=True),
        resources_per_trial={"cpu": cpu, "gpu": gpu},
        config=config,
        #num_samples=num_samples,
        scheduler=scheduler,
        verbose=1
        
    )
    return    
    best_trial = result.get_best_trial("loss", "min", "last")
    
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")
    

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint = best_trial.checkpoint.to_air_checkpoint()
    best_checkpoint_data = best_checkpoint.to_dict()
    best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])
    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))
    return best_trial.config
    

In [ ]:
def main():

    search_space = {
        #"opt": tune.grid_search(["Adam","SGD"]),
        "opt": "SGD",
        "l1": tune.grid_search([2**i for i in range(4,6)]),
        #"l2": tune.grid_search([2**i for i in range(4,7)]),
        "l2":16,
        #"lr": loguniform.rvs(1e-4, 1e-1, size=1)[0],#tune.loguniform(1e-4, 1e-1),
        "lr": 0.01,
        #"batch_size": tune.grid_search([32,64]),
        "batch_size": 32,
        #"dropout": np.random.uniform(low=.1, high=.3),
        "dropout": 0.2, 
    }
    

    best_config=hyperparam_search2(search_space,scheduler="ASHA",cpu=2,gpu=0,max_num_epochs=1,concurrent=0)


    print("--------training model--------")
    trainset, testset = load_data()
    net=train_cifar(best_config,epochs=200,trainset=trainset,testset=testset)
    test_acc = test_accuracy(net)
    print("Best trial test set accuracy: {}".format(test_acc))
    torch.save(net,'/working')

In [ ]:

if __name__ == "__main__":
    main()

Trial name,accuracy,date,done,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_cifar_41489_00000,0.628667,2023-06-01_01-05-13,True,11b6545873f0,1,1.10826,172.28.0.12,47538,True,39.3584,39.3584,39.3584,1685581513,1,41489_00000
train_cifar_41489_00001,0.655083,2023-06-01_01-05-19,True,11b6545873f0,1,1.09828,172.28.0.12,47595,True,34.1798,34.1798,34.1798,1685581519,1,41489_00001


2023-06-01 01:05:20,055	INFO tune.py:945 -- Total run time: 53.12 seconds (53.05 seconds for the tuning loop).


Best trial final validation loss: 1.0982751506169637
Best trial final validation accuracy: 0.6550833333333334
a:      {'opt': 'SGD', 'l1': 32, 'l2': 16, 'lr': 0.01, 'batch_size': 32, 'dropout': 0.2}


NameError: ignored